# 第1章 嵌入模型与向量数据库

# 学习目标

1. 理解向量表征  
2. 向量与文本向量  
3. 向量间的相似度计算  
4. Embedding Models嵌入模型原理及选择  
5. 向量数据库概述及核心原理  
6. Chroma向量数据库核心操作  
7. Milvus向量数据库扩展学习

# 1. 向量表征 (Vector Representation)

在人工智能领域，向量表征（Vector Representation）是核心概念之一。通过将文本、图像、声音、行为甚至复杂关系转化为高维向量（Embedding），AI系统能够以数学方式理解和处理现实世界中的复杂信息。这种表征方式为机器学习模型提供了统一的“语言”。

# 1.1. 向量表征的本质

向量表征的本质：万物皆可数学化

# (1) 核心思想

- 降维抽象：将复杂对象（如一段文字、一张图片）映射到低维稠密向量空间，保留关键语义或特征。  
- 相似性度量：向量空间中的距离(如余弦相似度)反映对象之间的语义关联(如"猫"和"狗"的向量距离小于"猫"和"汽车")。

# (2）数学意义

- 特征工程自动化：传统机器学习依赖人工设计特征（如文本的TF-IDF），而向量表征通过深度学习自动提取高阶抽象特征。  
- 跨模态统一：文本、图像、视频等不同模态数据可映射到同一向量空间,实现跨模态检索(如“用文字搜图片”)。

# 1.2. 向量表征的典型应用场景

(1) 自然语言处理 (NLP)

- 词向量 (Word2Vec、GloVe) : 单词映射为向量,解决“一词多义”问题(如“苹果”在“水果”和“公司”上下文中的不同向量)。  
- 句向量 (BERT、Sentence-BERT)：整句语义编码,用于文本相似度计算、聚类(如客服问答匹配)。  
- 知识图谱嵌入 (TransE、RotatE)：将实体和关系表示为向量,支持推理(如预测“巴黎-首都-法国”的三元组可信度)。

(2)计算机视觉 (CV)

- 图像特征向量 (CNN特征) : ResNet、ViT等模型提取图像语义,用于以图搜图、图像分类。  
- 跨模态对齐 (CLIP)：将图像和文本映射到同一空间,实现“描述文字生成图片”或反向搜索。

(3) 推荐系统

- 用户/物品向量：用户行为序列（点击、购买）编码为用户向量，商品属性编码为物品向量，通过向量内积预测兴趣匹配度（如YouTube推荐算法）。

(4) 复杂系统建模

- 图神经网络（GNN）：社交网络中的用户、商品、交互事件均表示为向量，捕捉网络结构信息（如社区发现、欺诈检测）。  
- 时间序列向量化：将股票价格、传感器数据编码为向量，预测未来趋势（如LSTM、Transformer编码）。

# 1.3. 向量表征的技术实现

# (1) 经典方法

- 无监督学习：Word2Vec通过上下文预测（Skip-Gram）或矩阵分解（GloVe）生成词向量。  
- 有监督学习：微调预训练模型（如BERT）适应具体任务，提取任务相关向量。

# (2) 前沿方向

- 对比学习 (Contrastive Learning)：通过构造正负样本对（如“同一图片的不同裁剪”为正样本），拉近正样本向量距离，推开负样本（SimCLR、MoCo）。  
- 多模态融合：将文本、图像、语音等多模态信息融合为统一向量（如Google的MUM模型）。  
- 动态向量：根据上下文动态调整向量（如Transformer的注意力机制），解决静态词向量无法适应多义性的问题


# 2. 什么是向量

向量是一种有大小和方向的数学对象。它可以表示为从一个点到另一个点的有向线段。例如，二维空间中的向量可以表示为  $(x, y)$ ，表示从原点  $(0, 0)$  到点  $(x, y)$  的有向线段。

<img src="https://cdn-mineru.openxlab.org.cn/result/2025-12-05/ec3da595-e9a3-4c52-9af1-2cc1dceffbd8/39021eecf01be2e027b1e6b1825f0f37742f03d68edeff7c32f633f05c8fe004.jpg" width="300px" height="200px"/>

<img src="https://cdn-mineru.openxlab.org.cn/result/2025-12-05/ec3da595-e9a3-4c52-9af1-2cc1dceffbd8/ad85ee07b2c4f8a05789b39866bc5f631a345c315aff74236aca5a890dc0e3f4.jpg" width="300px" height="200px"/>





# 2.1 文本向量 (Text Embeddings)

1. 将文本转成一组  $N$  维浮点数,即文本向量又叫 Embeddings  
2. 向量之间可以计算距离,距离远近对应语义相似度大小




<img src="https://cdn-mineru.openxlab.org.cn/result/2025-12-05/952d7419-7461-4b3b-a8a0-2db93fbddcd4/ce6eda81aa986ea969d14bf7afd7b6d659f991de7cfb32c4cdb6645fae93b371.jpg" width="300px" height="200px"/>

<img src="https://cdn-mineru.openxlab.org.cn/result/2025-12-05/952d7419-7461-4b3b-a8a0-2db93fbddcd4/149af6415027b15b63b982f04c662574aa1b97558dc40acebd1650b41bf4617a.jpg" width="300px" height="200px"/>

# 2.2 文本向量是怎么得到的（选）

1. 构建相关（正例）与不相关（负例）的句子对样本  
2. 训练双塔式模型,让正例间的距离小,负例间的距离大

例如：

<img src="https://cdn-mineru.openxlab.org.cn/result/2025-12-05/77a60418-8182-4687-839d-ba50c3708943/fdee8e636b61c4b2cd1eefb2b54fb7a8b1b4ad319ed59ebd854bec63c343267a.jpg" width="300px" height="200px"/>

<img src="https://cdn-mineru.openxlab.org.cn/result/2025-12-05/379ed34a-7245-4414-8e1b-20b072a360f5/d935a5b2338d28a62d85152b3c49d443c43267dca6cc91b626ebaede798e0d8e.jpg" width="300px" height="200px"/>


余弦相似度是通过计算两个向量夹角的余弦值来衡量相似性,等于两个向量的点积除以两个向量长度的乘积。

<table><tr><td>度量指标</td><td>定义</td><td>公式</td><td>取值范围</td><td>说明</td></tr><tr><td>点积</td><td>向量对应分量乘积之和</td><td>a·b = ∑i=1n ai b i</td><td>所有实数</td><td>值越大越相似。正数表示相同方向，负数反方向，0表示垂直</td></tr><tr><td>余弦相似度</td><td>衡量两个向量方向的相似程度</td><td>cosine(a, b) = a·b / ||a||·||b||</td><td>[-1, 1]</td><td>只感知方向，不涉及大小</td></tr><tr><td>欧几里得距离</td><td>两个向量在空间中的直线距离</td><td>d = √∑i=1n (ai - bi)2</td><td>[0, +∞)</td><td>值越小越相似。</td></tr></table>

In [ ]:
!pip install --upgrade numpy
!pip install --upgrade openai

In [1]:
import os
from openai import OpenAI

# 阿里百炼

client = OpenAI(
    api_key='sk-66bc27a6330f434f8751f8172a73064f',
    base_url='https://dashscope.aliyuncs.com/compatible-mode/v1'
)

print(client)

In [2]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [3]:

def cos_sim(a,b):
    '''余玄距离 -- 越大越相似'''
    return dot(a,b)/(norm(a)*norm(b))


def l2(a,b):
    '''欧式距离 -- 越小越相似'''
    x = np.asarray(a) -np.asarray(b)
    return norm(x)

In [4]:
def get_embeddings(texts,model='text-embedding-v1',dimensions=None):
    '''封装 OpenAi 的 Embedding 模型接口'''
    if model == 'text-embedding-v1':
        dimensions = None
    if dimensions:
        data = client.embeddings.create(
            input=texts,model=model,dimensions=dimensions
        ).data
    else:
        data = client.embeddings.create(
            input=texts,model=model
        ).data
    return [x.embedding for x in data]

In [5]:
test_query = ["聚客 Ai - 用科技力量，构建智能未来"]
vec = get_embeddings(test_query)[0]
print(f"total dimension : {len(vec)}")
print(f"first 10 elements : {vec[:10]}")

total dimension : 1536
first 10 elements : [1.462216854095459, -0.20184795558452606, 2.181633234024048, -1.5891999006271362, 3.3508434295654297, -3.247054100036621, 1.4340027570724487, -4.095310211181641, -3.479266881942749, 1.535062313079834]


In [ ]:
query = "国际争端"

documents = [
    "联合国就苏丹达尔富尔地区大规模暴力事件发出警告",
    "土耳其、芬兰、瑞典与北约代表将继续就瑞典'入北约'问题进行谈判",
    "日本支付服饰陆上自卫队射击场内发生枪击事件，3 人受伤", 
    "国家游泳中心（水立方）：恢复游泳、嬉水乐园等水上项目运营", 
    "我国首次在空间开展舱外辐射生物学暴露实验",  
]

quert_vec = get_embeddings([query])[0]
docs_vecs = get_embeddings(documents)

print("Query 与自己的余弦距离: {:.2}".format(cos_sim(quert_vec,quert_vec)))
print("Query 与 document的余弦距离:")
for vec in docs_vecs:
    print(cos_sim(quert_vec,vec))

print()

print("Query 与自己的欧式距离: {:.2}".format(l2(quert_vec,quert_vec)))
print("Query 与 document的欧式距离:")
for vec in docs_vecs:
    print(l2(quert_vec,vec))



# 3. Embedding Models 嵌入模型

# 3.1. 什么是嵌入（Embedding）？

嵌入（Embedding）是指非结构化数据转换为向量的过程，通过神经网络模型或相关大模型，将真实世界的离散数据投影到高维数据空间上，根据数据在空间中的不同距离，反映数据在物理世界的相似度。

# 3.2. 嵌入模型概念及原理

# 1. 嵌入模型的本质

嵌入模型（Embedding Model）是一种将离散数据（如文本、图像）映射到连续向量空间的技术。通过高维向量表示（如768维或3072维），模型可捕捉数据的语义信息，使得语义相似的文本在向量空间中距离更近。例如，“忘记密码”和“账号锁定”会被编码为相近的向量，从而支持语义检索而非仅关键词匹配。

# 2.核心作用

- 语义编码：将文本、图像等转换为向量，保留上下文信息（如BERT的CLS Token或均值池化。相似度计算：通过余弦相似度、欧氏距离等度量向量关联性，支撑检索增强生成（RAG）、推荐系统等应用。  
- 信息降维：压缩复杂数据为低维稠密向量，提升存储与计算效率。

# 3.关键技术原理

- 上下文依赖：现代模型（如BGE-M3）动态调整向量，捕捉多义词在不同语境中的含义。  
- 训练方法：对比学习（如Word2Vec的Skip-gram/CBOW）、预训练+微调（如BERT）。

# 3.3.主流嵌入模型分类与选型指南

- Embedding 模型将文本转换为数值向量，捕捉语义信息，使计算机能够理解和比较内容的“意义”。选择 Embedding 模型的考虑因素：

<table><tr><td>因素</td><td>说明</td></tr><tr><td>任务性质</td><td>匹配任务需求(问答、搜索、聚类等)</td></tr><tr><td>领域特性</td><td>通用vs专业领域(医学、法律等)</td></tr><tr><td>多语言支持</td><td>需处理多语言内容时考虑</td></tr><tr><td>维度</td><td>权衡信息丰富度与计算成本</td></tr><tr><td>许可条款</td><td>开源vs专有服务</td></tr><tr><td>最大Tokens</td><td>适合的上下文窗口大小</td></tr></table>

最佳实践：为特定应用测试多个 Embedding 模型，评估在实际数据上的性能而非仅依赖通用基准。

# 1. 通用全能型

- BGE-M3：北京智源研究院开发，支持多语言、混合检索（稠密+稀疏向量），处理8K上下文，适合企业级知识库。  
- NV-Embed-v2：基于Mistral-7B，检索精度高（MTEB得分62.65），但需较高计算资源。

      稠密向量（Dense Vector） 和 稀疏向量（Sparse Vector） 是两种核心的向量表示形式，二者的核心区别在于向量维度的非零元素占比和语义编码方式。BGE-M3 支持的 混合检索，就是同时利用这两种向量的优势提升检索效果
      稠密向量是指绝大多数维度的值都不为零的向量，维度通常是固定的（如 768、1024、2048 维），每个维度的值是连续的浮点数（如 0.123, -0.456, 0.789）。
      由深度学习模型（如 Transformer 架构的 BERT、Sentence-BERT、BGE 等）生成，模型通过大量语料学习文本的语义关联，将相似语义的内容映射到语义空间的相近位置。
      优势：擅长捕捉文本的深层语义关联，比如同义词、语义相似但字面不同的内容（如 “手机没电了” 和 “移动设备电量耗尽”），检索的语义匹配精度高。
      不足：向量维度高，存储和计算成本较高；对字面差异大但语义相关的内容依赖模型训练效果；无法直接体现关键词的权重。

  
      稀疏向量是指绝大多数维度的值为零，只有少数维度有非零值的向量，维度通常非常高（甚至可达几十万到上百万维，对应词汇表大小），非零维度的值通常是离散的权重（如 TF-IDF 权重、BM25 分数）。它的维度与词汇表一一对应，每个维度代表一个具体的词汇，非零值的位置对应文本中出现的词汇，数值代表该词汇的重要性。
      基于统计规则生成，无需深度学习模型，常见方法包括：

      TF-IDF：衡量词汇在文本中的重要性（词频 × 逆文档频率）。
      BM25：改进的 TF-IDF 算法，更适合检索场景。
      词袋模型（Bag of Words）：简单统计词汇出现次数。

        
      优势：向量稀疏性高，存储和计算成本低（可通过稀疏矩阵优化）；直接体现关键词的匹配，对含特定关键词的内容检索效率高；可解释性强（非零维度对应具体词汇）。
      不足：无法捕捉深层语义关联，容易受字面差异影响（如 “手机” 和 “移动电话” 会被视为不同维度）；对长文本的权重分配不够精准。


      混合检索（稠密 + 稀疏向量）的价值
      BGE-M3 支持的混合检索，就是同时生成文本的稠密向量和稀疏向量，检索时融合两种向量的匹配分数，兼顾语义精度和关键词匹配效率，核心优势如下：

      互补短板：稠密向量解决 “语义相似但字面不同” 的检索问题，稀疏向量解决 “关键词精准匹配” 的需求，结合后检索召回率和准确率更高。
      适配多场景：既适合知识库中语义化的问答检索（如 “如何配置 Spring 事务”），也适合关键词精准检索（如 “BGE-M3 8K 上下文”）。
      企业级实用性：8K 长上下文支持可处理文档、手册等长文本，混合检索的效率和精度平衡，更适合企业知识库的大规模部署。


# 2.垂直领域特化型

- 中文场景：BGE-large-zh-v1.5（合同/政策文件）、M3E-base（社交媒体分析）。  
- 多模态场景：BGE-VL（图文跨模态检索），联合编码OCR文本与图像特征。

# 3. 轻量化部署型

- nomic-embed-text: 768 维向量, 推理速度比 OpenAI 快 3 倍, 适合边缘设备。  
- gte-qwen2-1.5b-instruct: 1.5B 参数, 16GB 显存即可运行, 适合初创团队原型验。

# 选型决策树：

1. 中文为主  $\rightarrow$  BGE 系列  $>\mathrm{M}3\mathrm{E}$  
2. 多语言需求  $\rightarrow$  BGE-M3 > multilingual-e5;  
3. 预算有限  $\rightarrow$  开源模型（如Nomic Embed）



# Embuddding Leaderboard

<table><tr><td>Rank (Borda)</td><td>Model</td><td>Zero-shot</td><td>Memory Usage (MB)</td><td>Number of Parameters</td><td>Embedding Dimensions</td><td>Max Tokens</td></tr><tr><td>1</td><td>gemini-embedding-exp-03-07</td><td>99%</td><td>Unknown</td><td>Unknown</td><td>3072</td><td>8192</td></tr><tr><td>2</td><td>Ling-Embed-Mistral</td><td>99%</td><td>13563</td><td>78</td><td>4096</td><td>32768</td></tr><tr><td>3</td><td>gte-Qwen2-78-instruct</td><td>▲ NA</td><td>29040</td><td>78</td><td>3584</td><td>32768</td></tr><tr><td>4</td><td>multilingual-e5-large-instruct</td><td>99%</td><td>1068</td><td>560M</td><td>1024</td><td>514</td></tr><tr><td>5</td><td>SFR-Embedding-Mistral</td><td>96%</td><td>13563</td><td>78</td><td>4096</td><td>32768</td></tr><tr><td>6</td><td>GritLM-7B</td><td>99%</td><td>13813</td><td>78</td><td>4096</td><td>4096</td></tr><tr><td>7</td><td>text-multilingual-embedding-002</td><td>99%</td><td>Unknown</td><td>Unknown</td><td>768</td><td>2048</td></tr><tr><td>8</td><td>GritLM-8x7B</td><td>99%</td><td>89079</td><td>578</td><td>4096</td><td>4096</td></tr><tr><td>9</td><td>e5-mistral-7b-instruct</td><td>99%</td><td>13563</td><td>78</td><td>4096</td><td>32768</td></tr></table>

<table><tr><td>Mean (Task)</td><td>Mean (TaskType)</td><td>Bitext1 Mining</td><td>Classification</td></tr><tr><td>68.32</td><td>59.64</td><td>79.28</td><td>71.82</td></tr><tr><td>61.47</td><td>54.21</td><td>70.34</td><td>62.24</td></tr><tr><td>62.51</td><td>56.00</td><td>73.92</td><td>61.55</td></tr><tr><td>63.23</td><td>55.17</td><td>80.13</td><td>64.94</td></tr><tr><td>60.93</td><td>54.00</td><td>70.00</td><td>60.02</td></tr><tr><td>60.93</td><td>53.83</td><td>70.53</td><td>61.83</td></tr><tr><td>62.13</td><td>54.32</td><td>70.73</td><td>64.64</td></tr><tr><td>60.50</td><td>53.39</td><td>68.17</td><td>61.55</td></tr><tr><td>60.28</td><td>53.18</td><td>70.58</td><td>60.31</td></tr></table>

https://huggingface.co/spaces/mteb/leaderboard
 

In [ ]:
import os
from openai import OpenAI

# 阿里百炼

client = OpenAI(
    api_key='sk-66bc27a6330f434f8751f8172a73064f',
    base_url='https://dashscope.aliyuncs.com/compatible-mode/v1'
)

completion = client.embeddings.create(
    model="text-embedding-v4",
    input="聚客 AI",
    dimensions=1024,encoding_format="float"
)

print(completion.model_dump_json())


# 4. 向量数据库

# 4.1. 什么是向量数据库？

向量数据库,是专门为向量检索设计的中间件！

高效存储、快速检索和管理高纬度向量数据的系统称为向量数据库

一种专门用于存储和检索高维向量数据的数据库。它将数据（如文本、图像、音频等）通过嵌入模型转换为向量形式，并通过高效的索引和搜索算法实现快速检索。

向量数据库的核心作用是实现相似性搜索，即通过计算向量之间的距离（如欧几里得距离、余弦相似度等）来找到与目标向量最相似的其他向量。它特别适合处理非结构化数据，支持语义搜索、内容推荐等场景。

# 核心功能：

- 向量存储  
- 相似性度量  
- 相似性搜索

# 4.2. 如何存储和检索嵌入向量？

- 存储：向量数据库将嵌入向量存储为高维空间中的点，并为每个向量分配唯一标识符（ID），同时支持存储元数据。  
- 检索：通过近似最近邻（ANN）算法（如PQ等）对向量进行索引和快速搜索。比如，FAISS和Milvus等数据库通过高效的索引结构加速检索。


# 4.3 向量数据库与传统数据库对比

1. 数据类型

- 传统数据库：存储结构化数据（如表格、行、列）。  
- 向量数据库：存储高维向量数据，适合非结构化数据。

2. 查询方式

- 传统数据库：依赖精确匹配（如=、<、>）。  
- 向量数据库：基于相似度或距离度量（如欧几里得距离、余弦相似度）。

3. 应用场景

- 传统数据库：适合事务记录和结构化信息管理。  
- 向量数据库：适合语义搜索、内容推荐等需要相似性计算的场景。

# 澄清几个关键概念：

- 向量数据库的意义是快速的检索；  
- 向量数据库本身不生成向量,向量是由 Embedding 模型产生的；  
- 向量数据库与传统的关系型数据库是互补的,不是替代关系,在实际应用中根据实际需求经常同时使用。

# 4.4.主流向量数据库功能对比

<table><tr><td>Name</td><td>Web GUI</td><td>GPU Support</td><td>Remote Support (HTTP/gRPC)</td><td>Cloud Native</td><td>Opensource</td><td>Metadata (hybrid search)</td></tr><tr><td>FAISS</td><td>N</td><td>Y</td><td>N</td><td>N</td><td>Y</td><td>N</td></tr><tr><td>Pinecone</td><td>Y</td><td>N/A</td><td>Y</td><td>Y</td><td>N</td><td>Y</td></tr><tr><td>Milvus</td><td>Y</td><td>Y</td><td>Y</td><td>Y</td><td>Y</td><td>Y</td></tr><tr><td>Weaviate</td><td>N</td><td>N I</td><td>Y</td><td>Y</td><td>Y</td><td>Y</td></tr><tr><td>Qdrant</td><td>N</td><td>Y</td><td>Y</td><td>Y</td><td>Y</td><td>Y</td></tr><tr><td>PGVector</td><td>N</td><td>N</td><td>Y</td><td>Y</td><td>Y</td><td>Y</td></tr><tr><td>RediSearch</td><td>N</td><td>N</td><td>Y</td><td>Y</td><td>Y</td><td>Y</td></tr></table>

- FAISS: Meta 开源的向量检索引擎 https://github.com/facebookresearch/faiss  
- Pinecone: 商用向量数据库, 只有云服务 https://www.pinecone.io/  
- Milvus: 开源向量数据库,同时有云服务 https://milvus.io/  
- Weaviate: 开源向量数据库, 同时有云服务 https://weaviate.io/


# 1. 什么是 Chroma?

Chroma 是一款开源的向量数据库，专为高效存储和检索高维向量数据设计。其核心能力在于语义相似性搜索，支持文本、图像等嵌入向量的快速匹配，广泛应用于大模型上下文增强（RAG）、推荐系统、多模态检索等场景。与传统数据库不同，Chroma 基于向量距离（如余弦相似度、欧氏距离）衡量数据关联性，而非关键词匹配。

# 2.核心优势

- 轻量易用：以Python/JS包形式嵌入代码，无需独立部署，适合快速原型开发。  
- 灵活集成：支持自定义嵌入模型（如OpenAI、HuggingFace），兼容LangChain等框架。  
- 高性能检索：采用 HNSW 算法优化索引，支持百万级向量毫秒级响应。  
- 多模式存储：内存模式用于开发调试，持久化模式支持生产环境数据落地。


扩展阅读:https://guangzhengli.com/blog/zh/vector-database


# 4.6. Chroma 安装与基础配置


In [ ]:
#!pip install chromadb

In [1]:
import chromadb

In [9]:
# 临时
client = chromadb.EphemeralClient()
print(client)

In [2]:
# 持久化运行
client = chromadb.PersistentClient('./chromadb')

print(client)

# 1. 集合 (Collection)

集合是 Chroma 中管理数据的基本单元,类似关系数据库的表：

(1) 创建集合


In [3]:
from chromadb.utils import embedding_functions
# 默认情况下，Chroma 使用 DefaultEmbeddingFunction，它是基于 Sentence Transformers 的 MiniLM-L6-v2 模型
default_ef = embedding_functions.DefaultEmbeddingFunction()
# 使用 OpenAI 的嵌入模型，默认使用 text-embedding-ada-002 模型
#openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#    api_key="YOUR_API_KEY",
#    model_name="text-embedding-3-small"
# )

In [4]:
#自定义 Embedding Functions 
from chromadb import Documents, EmbeddingFunction, Embeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, texts: Documents) -> Embeddings: 
        # embed the documents somehow 
        return embeddings

In [5]:

collection = client.create_collection(
    name = "my_collection",  
    configuration = {
    #HNSW 索引算法，基于图的近似最近邻搜索算法（Approximate Nearest Neighbor, ANN）
    "hnsw": {
        "space": "cosine", # 指定余弦相似度计算
        "ef_search": 100,
        "ef_construction": 100,
        "max_neighbors": 16,
        "num_threads": 4
    },
    #指定向量模型
    "embedding_function": default_ef
    }
)


## (2)查询集合

In [6]:
collection = client.get_collection(name="my_collection")

- peek() - returns a list of the first 10 items in the collection.  
- count() - returns the number of items in the collection.  
- modify() - rename the collection

In [7]:
print(collection.peek())

{'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['metadatas', 'documents', 'embeddings'], 'data': None, 'metadatas': []}


In [10]:
print(collection.count())

3


##  (3) 删除集合

In [ ]:
# client.delete_collection(name="")

# 2.添加数据

In [9]:
# 方式1：自动生成向量（使用集合指定的嵌入模型）

collection.add(
   documents=["RAG是一种检索增强生成技术",
              "向量数据库有存储文档的嵌入式表示",
              "在机器学习领域，智能体（Agent）通常能够感知知识"],
    metadatas=[{"source":"RAG"},{"source":"向量数据库"},{"source":"Agent"}],
    ids=["id1","id2","id3"]
)

# 方式2：手动传入预计算向量
# collection.add(
#     embeddings=[[0.322,0.34234],[0.3242,0.424],[0.2323,0.3232]],
#     documents=[{"source":"RAG"},{"source":"向量数据库"},{"source":"Agent"}],
#     ids=["id1","id2","id3"]
# )




/Users/lojzes/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|███████| 79.3M/79.3M [01:00<00:00, 1.37MiB/s]


# 3. 查询数据

In [14]:
result = collection.query(
     query_texts=["RAG 是什么？"],
     n_results=3,
     # where={"source":"RAG"},
     # where_document={"$contains":"检索增强生成"}  
)
print(result)

{'ids': [['id1']], 'embeddings': None, 'documents': [['RAG是一种检索增强生成技术']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'source': 'RAG'}]], 'distances': [[0.2897658348083496]]}


# 向量查询（自定义输入）

In [ ]:
result = collection.query(
     query_embeddings=[[0.3232,0.432432]],
     n_results=3,
)
print(result)

# 4 数据管理

更新集合中的数据

In [19]:
collection.update(ids=["id1"],documents=["RAG是一种检索增强生成技术,在智能客服系统中大量使用"])

In [25]:
print(collection.query(query_texts=["RAG 是什么？"],
     n_results=3,
     # where={"source":"RAG"}
                      )
     )

{'ids': [['id1', 'id2', 'id3']], 'embeddings': None, 'documents': [['RAG是一种检索增强生成技术,在智能客服系统中大量使用', '向量数据库有存储文档的嵌入式表示', '在机器学习领域，智能体（Agent）通常能够感知知识']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'source': 'RAG'}, {'source': '向量数据库'}, {'source': 'Agent'}]], 'distances': [[0.3798767328262329, 0.5413401126861572, 0.6665253639221191]]}


# 删除数据

In [26]:
collection.delete(ids=["id3"])

# 4.8 Chroma Clinet-Server Mode

- Server 端



# 在命令行中执行

In [31]:
chroma run --path ./server_chromadb

SyntaxError: invalid syntax (2089067113.py, line 1)

- Client 端  

In [32]:
import chromadb

chroma_client = chromadb.HttpClient(host="localhost",port=8000)

print(chroma_client)


# Milvus 扩展学习

## 中文官网 : https://milvus.io/zh